# Galfitting with lenstronomy

- https://colab.research.google.com/github/lenstronomy/lenstronomy-tutorials/blob/main/Notebooks/Galaxies/galfitting_with_lenstronomy.ipynb

An example of using **lenstronomy** without imposing a lensing deflector is the inference of galaxy structural parameters through the fitting of a parameterized surface brightness model (or many thereof).

We want to demonstrate the flexibility of **lenstronomy**. Matching the structural properties of lensing and source galaxy is an integral part of lens modelling. **lenstronomy** may provide an alternative in python to other software that are more specific to this task.
The 'lens_light_model' effectively describes undistorted surface brightness profiles in lenstronomy.

**lenstronomy** supports a wide range of light profiles. You can find the current list here: https://github.com/lenstronomy/lenstronomy/blob/main/lenstronomy/LightModel/light_model_base.py

A key difference to other ligth profile fitting codes is that **lenstronomy** solves the amplitude parameter of the different components with a linear minimizer on the fly. An arbitrary number of superposition of different profiles is supported, including shapelets and wavelets, to describe complex structure.

**lenstronomy** also supports multi-band, multi-exposure, and multi-object fitting. We refer to other notebooks about the handling of those tasks. In short, all the tasks that are supported when using lenstronomy in the 'lensing model' are also supported when turning lensing off - with the identical API.

<div class="alert alert-warning">

**Warning:** Small sources and very cuspy cores (such as Sersic profiels with high Sersic indexes) result in steep surface brightness variations below the pixel scale. The numerics used in this notebook only evaluates the surface brighness in the centre of each pixel and assigns the entire pixel this surface brightness value. Super-sampled evaluation of surface brightness profiles may be required. Lenstronomy supports this, both on a grid and adaptive in specific regions. We refer to the $\texttt{Numerics}$ module and a dedicated notebook on numerical aspects.

</div>

Additional features are available with the software [GaLight](https://github.com/dartoon/galight), which wrapps around lenstronomy.


In [ ]:
# some standard python imports #
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# make sure lenstronomy is installed, otherwise install the latest pip version
try:
    import lenstronomy
except:
    !pip install lenstronomy

## create mock image as a superposition of different profiles
In the example below, we generate a galaxy image with a two component Sersic profile at the same center and a compagnion Hernquist light distribution. The data quality is comparable with wide field surveys.

In [ ]:
import lenstronomy.Util.simulation_util as sim_util
import lenstronomy.Util.image_util as image_util
from lenstronomy.Data.imaging_data import ImageData
from lenstronomy.Data.psf import PSF

In [ ]:
# data specifics
background_rms = .05  # background noise per pixel
exp_time = 100  # exposure time (arbitrary units, flux per pixel is in units #photons/exp_time unit)
numPix = 50  # cutout pixel size
deltaPix = 0.3  # pixel size in arcsec (area per pixel = deltaPix**2)
fwhm = 0.8  # full width half max of PSF

In [ ]:
kwargs_data = sim_util.data_configure_simple(numPix, deltaPix, exp_time, background_rms)
data_class = ImageData(**kwargs_data)


In [ ]:
# PSF specification
kwargs_psf = {'psf_type': 'GAUSSIAN', 'fwhm': fwhm, 'pixel_size': deltaPix, 'truncation': 6}
psf_class = PSF(**kwargs_psf)

In [ ]:
# create a model with three Sersic profiles
# all the models are part of 'lens_light_model_list', meaning that their surface brightness profile are not lensed
lens_light_model_list = ['SERSIC_ELLIPSE', 'SERSIC_ELLIPSE', 'HERNQUIST']
from lenstronomy.LightModel.light_model import LightModel
lightModel = LightModel(lens_light_model_list)

In [ ]:
kwargs_1 = {'amp': 100, 'R_sersic': .5, 'n_sersic': 3, 'e1': 0.1, 'e2': 0, 'center_x': 0, 'center_y': 0}
kwargs_2 = {'amp': 100, 'R_sersic': 1.5, 'n_sersic': 1, 'e1': 0.2, 'e2': -0.2, 'center_x': 0, 'center_y': 0}
kwargs_3 = {'amp': 100, 'Rs': 0.3, 'center_x': 3.5, 'center_y': -0.5}
kwargs_light = [kwargs_1, kwargs_2, kwargs_3]

In [ ]:
# here we super-sample the resolution of some of the pixels where the surface brightness profile has a high gradient 
supersampled_indexes = np.zeros((numPix, numPix), dtype=bool)
supersampled_indexes[23:27, 23:27] = True

In [ ]:
kwargs_numerics = {'supersampling_factor': 4, 
                   'compute_mode': 'adaptive',
                  'supersampled_indexes': supersampled_indexes}
from lenstronomy.ImSim.image_model import ImageModel
imageModel = ImageModel(data_class, psf_class, lens_light_model_class=lightModel, kwargs_numerics=kwargs_numerics)
image_sim = imageModel.image(kwargs_lens_light=kwargs_light)
poisson = image_util.add_poisson(image_sim, exp_time=exp_time)
bkg = image_util.add_background(image_sim, sigma_bkd=background_rms)
image_noisy = image_sim + bkg + poisson
data_class.update_data(image_noisy)
kwargs_data['image_data'] = image_noisy

In [ ]:
plt.matshow(np.log10(image_noisy), origin='lower')
plt.show()